# Experiments with Taiwan

In [1]:
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from sklearn.inspection import PartialDependenceDisplay
from lime import lime_tabular
import shap
from cfmining.algorithms import MAPOCAM, BruteForce, Greedy
from cfmining.criteria import PercentileCalculator, PercentileCriterion, PercentileChangesCriterion, NonDomCriterion
from cfmining.predictors import MonotoneClassifier
from cfmining.visualization import buildTable, PlotCounterfactuals
from cfmining.mip_builder import RecourseBuilder
from cfmining.action_set import ActionSet
import joblib


import credit_pipeline.data_exploration as dex
from credit_pipeline.training import *


%load_ext autoreload
%autoreload 2

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Loading and Cleaning

In [2]:
seed_number = 0

In [64]:
path = "../data/Taiwan/"
df = dex.read_csv_encoded(path, 'Taiwan.csv')
df.columns = df.iloc[0, :].tolist()
df = df.iloc[1:, :]
df = df.drop(columns = ["ID"])

In [65]:
columns_to_drop = dex.check_missing(df, 50,  False)
columns_to_drop

[]

In [66]:
num_cols = ["LIMIT_BAL", "AGE", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3",
            "BILL_AMT4", "BILL_AMT5", "BILL_AMT6", "PAY_AMT1", "PAY_AMT2",
            "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6", "default payment next month"]
df[num_cols] = df[num_cols].astype(float)

In [67]:
df_cols = df.columns.to_list()
obj_cols = dex.list_by_type(df, ['O'])
obj_cols

['SEX',
 'EDUCATION',
 'MARRIAGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6']

In [68]:
print(dex.list_no_variation_cols(df))
print(dex.list_contin_cols(df))
print(dex.list_by_unique(df, 2))

[]
['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
['SEX', 'default payment next month']


In [71]:
df["SEX"] = df["SEX"].apply(lambda x : "female" if x == "2" else "male")

## Training Basic Models

In [72]:
X_acp = df.iloc[:, (df.columns != "default payment next month")]
y_acp = df["default payment next month"]

In [73]:
X_train, X_test, y_train, y_test = train_test_split(
    X_acp, 
    y_acp, 
    test_size = 0.2, 
    random_state=seed_number, 
    stratify=y_acp
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, 
    y_train, 
    test_size = 0.2, 
    random_state=seed_number, 
    stratify=y_train
)

In [74]:
classifiers = {
    "Logistic Regression": LogisticRegression,
    "Random Forest": RandomForestClassifier,
    "LightGBM": LGBMClassifier,
    "MLPC" : MLPClassifier,
}

In [75]:
param_spaces = {
    "LogisticRegression": {
        'C': {'low': 0.001, 'high': 10, 'log': True, 'type':'float'},
        'max_iter': {'low': 1000, 'high': 1000, 'step':1, 'type':'int'},
        'penalty': {'choices': ['l2'], 'type':'categorical'},
        "class_weight" : {"choices" : [None, "balanced"], 'type':'categorical'},
    },
    "RandomForestClassifier": {
        'n_estimators': {'low':10, 'high':150, 'step':20, 'type':'int'},
        'max_depth': {'low':2, 'high':10, 'type':'int'},
        'criterion': {'choices':['gini', 'entropy'], 'type':'categorical'},
        'min_samples_leaf' : {"low" : 1, "high" : 51, "step" : 5, 'type':'int'},
        "max_features" : {"low" : 0.1, "high" : 1.0, "type" : "float"},
        "class_weight" : {"choices" : [None, "balanced"], 'type':'categorical'},
    },
    "LGBMClassifier": {
        'learning_rate': {'low': 0.01, 'high': 1.0, 'type': 'float', 'log': True},
        "num_leaves" : {"low" : 5, "high" : 100, "step" : 5, 'type':'int'},
        'max_depth': {'low': 2, 'high': 10, 'type': 'int'},
        'min_child_samples': {'low': 1, 'high': 51, 'step': 5, 'type': 'int'},
        'colsample_bytree': {'low': 0.1, 'high': 1.0, 'type': 'float'},
        'reg_alpha': {'low': 0.0, 'high': 1.0, 'type': 'float'},
        'reg_lambda': {'low': 0.0, 'high': 1.0, 'type': 'float'},
        'n_estimators': {'low': 10, 'high': 100, 'step': 10, 'type': 'int'},
        "class_weight" : {"choices" : [None, "balanced"], 'type':'categorical'},
        "verbose" : {"choices" : [-1], 'type':'categorical'},

    },
    "MLPClassifier": {
        "hidden_layer_sizes" : {"choices" : [
            [128, 64, 32],
            [128, 64, 32, 16],
            [256, 128, 64, 32, 16],
        ], 'type':'categorical'},
        "alpha" : {'low': 0.0001, 'high': 0.01, 'type': 'float', 'log': True},
        "learning_rate" : {'choices': ['constant', 'invscaling', 'adaptive'], 'type':'categorical'},
        "learning_rate_init" : {'low': 0.001, 'high': 0.1, 'type': 'float', 'log': True},
        "early_stopping" : {'choices': [True], 'type':'categorical'},
        "max_iter" : {"choices" : [50], 'type':'categorical'},
    }
}

In [ ]:
study_logistic, model_logistic = optimize_model(LogisticRegression, param_spaces["LogisticRegression"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_logistic, "models/taiwan_logistic.joblib")

In [77]:
print("Score for Logistic Regression: ", study_logistic.best_value)
print(study_logistic.best_params)

Score for Logistic Regression:  0.774217861248903
{'C': 0.007270375039865594, 'max_iter': 1000, 'penalty': 'l2', 'class_weight': None}


In [ ]:
study_rf, model_rf = optimize_model(RandomForestClassifier, param_spaces["RandomForestClassifier"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_rf, "models/taiwan_rf.joblib")

In [79]:
print("Score for Random Forest: ", study_rf.best_value)
print(study_rf.best_params)

Score for Random Forest:  0.7854082215632396
{'n_estimators': 70, 'max_depth': 10, 'criterion': 'entropy', 'min_samples_leaf': 41, 'max_features': 0.6223181749192774, 'class_weight': 'balanced'}


In [ ]:
study_lgbm, model_lgbm = optimize_model(LGBMClassifier, param_spaces["LGBMClassifier"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_lgbm, "models/taiwan_lgbm.joblib")

In [81]:
print("Score for LGBM: ", study_lgbm.best_value)
print(study_lgbm.best_params)

Score for LGBM:  0.7877014355542254
{'learning_rate': 0.05699910588443634, 'num_leaves': 45, 'max_depth': 10, 'min_child_samples': 16, 'colsample_bytree': 0.3234057672082941, 'reg_alpha': 0.45527006502059564, 'reg_lambda': 0.9669089166677883, 'n_estimators': 80, 'class_weight': 'balanced', 'verbose': -1}


In [ ]:
study_mlp, model_mlp = optimize_model(MLPClassifier, param_spaces["MLPClassifier"], X_train, y_train, X_val , y_val, n_trials=100)
joblib.dump(model_mlp, "models/taiwan_mlp.joblib")

In [83]:
print("Score for MLP: ", study_mlp.best_value)
print(study_mlp.best_params)

Score for MLP:  0.7859573737025651
{'hidden_layer_sizes': [256, 128, 64, 32, 16], 'alpha': 0.0007524655395096057, 'learning_rate': 'constant', 'learning_rate_init': 0.0025686779592416402, 'early_stopping': True, 'max_iter': 50}


## Model evaluation

In [84]:
model_logistic = joblib.load("models/taiwan_logistic.joblib")
model_rf = joblib.load("models/taiwan_rf.joblib")
model_lgbm = joblib.load("models/taiwan_lgbm.joblib")
model_mlp = joblib.load("models/taiwan_mlp.joblib")
models_dict = {
    "Logistic Regression" : 
        [model_logistic, ks_threshold(y_test, model_logistic.predict_proba(X_test)[:,1])],
    "Random Forest" : 
        [model_rf, ks_threshold(y_test, model_rf.predict_proba(X_test)[:,1])],
    "LightGBM" : 
        [model_lgbm, ks_threshold(y_test, model_lgbm.predict_proba(X_test)[:,1])],
    "MLP" : 
        [model_mlp, ks_threshold(y_test, model_mlp.predict_proba(X_test)[:,1])],
}

In [85]:
get_metrics(models_dict, X_test, y_test)

,AUC,Balanced Accuracy,Accuracy,Precision,Recall,F1,Brier Score
Logistic Regression,0.761410,0.700887,0.759333,0.465568,0.596081,0.522802,0.137477
Random Forest,0.781237,0.714317,0.770167,0.484542,0.614167,0.541708,0.173717
LightGBM,0.786544,0.715759,0.772833,0.489183,0.613414,0.544300,0.175055
MLP,0.760713,0.700267,0.734833,0.432584,0.638282,0.515677,0.140906


In [86]:
get_fairness_metrics(models_dict, X_test, y_test, X_test.SEX == "female")

,SPD,EOD,GMA
Logistic Regression,-0.054048,-0.024142,0.755688
Random Forest,-0.034629,-0.003833,0.767718
LightGBM,-0.032769,0.000918,0.770350
MLP,-0.091951,-0.058162,0.729030


## Explainability

**TODO**

*   Add easy way to include default parameters in optuna
*   Warnings on pipeline given numeric unique number of values
